# vLLM部署ERNIE-4.5-0.3B教程

本教程将详细介绍如何使用vLLM部署百度ERNIE-4.5-0.3B模型，实现高性能的推理服务。ERNIE-4.5是百度发布的新一代大语言模型，具有优秀的中文理解和生成能力。vLLM是一个高性能的大语言模型推理引擎，专为生产环境设计，支持高吞吐量的批量推理和在线服务。

## 环境准备

### 硬件要求

- **GPU**: 推荐RTX 4090 24G（ERNIE-4.5-0.3B模型实际只需要2-4GB显存，但RTX 4090可以提供更好的推理性能）
- **内存**: 至少16GB RAM（推荐32GB以获得更好的性能）
- **存储**: 至少10GB可用空间（用于模型文件和缓存）
- **操作系统**: Linux（推荐Ubuntu 20.04+）

### 软件要求

- **Python**: 3.9-3.12（推荐3.10）
- **CUDA**: 11.8+（如果使用GPU）
- **PyTorch**: 2.0+
- **网络**: 稳定的网络连接（首次运行时需要下载模型）

In [1]:
# 检查Python版本
!python --version

Python 3.10.18


In [ ]:
# 检查CUDA版本
!nvidia-smi

Thu Aug 21 15:25:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
|  0%   46C    P8             20W /  450W |    2125MiB /  24564MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# 检查可用显存
!nvidia-smi --query-gpu=memory.total,memory.free --format=csv

memory.total [MiB], memory.free [MiB]
24564 MiB, 22029 MiB


## vLLM安装

vLLM的安装需要特别注意版本兼容性，因为ERNIE-4.5模型支持是在较新版本中添加的。

### 创建虚拟环境

```bash
# 创建conda环境（推荐使用Python 3.10）
conda create -n vllm-ernie python=3.10 -y
conda activate vllm-ernie

# 更新pip到最新版本
pip install --upgrade pip
```

### 安装vLLM

```bash
# 安装vLLM（确保使用最新版本以支持ERNIE模型）
pip install vllm

```

### 验证安装

In [4]:
import vllm
print(f"vLLM version: {vllm.__version__}")

# 检查CUDA是否可用
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")

vLLM version: 0.10.1
CUDA available: True
CUDA device count: 1
Current CUDA device: 0
CUDA device name: NVIDIA GeForce RTX 4090


### 常见安装问题

1. **CUDA版本不匹配**: 确保CUDA版本与PyTorch版本兼容
2. **内存不足**: 安装过程中可能需要较多内存，建议关闭其他程序
3. **网络问题**: 如果下载缓慢，可以使用国内镜像源

## SDK调用示例

### 基础文本生成

以下示例展示了如何使用vLLM SDK进行基础的文本生成：

In [1]:
import os
# huggingface 镜像地址
os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"

from vllm import LLM, SamplingParams
import time


# 初始化模型
print("正在加载ERNIE-4.5-0.3B模型...")
start_time = time.time()

llm = LLM(
    model="baidu/ERNIE-4.5-0.3B-PT",  # 使用PyTorch版本的ERNIE模型
    trust_remote_code=True,           # 必须设置，允许执行自定义代码
    gpu_memory_utilization=0.8,      # GPU内存使用率
    max_model_len=2048,               # 最大序列长度
    dtype="float16"                   # 使用半精度以节省显存
)

load_time = time.time() - start_time
print(f"模型加载完成，耗时: {load_time:.2f}秒")

# 设置采样参数
sampling_params = SamplingParams(
    temperature=0.7,      # 控制生成的随机性，0.0-2.0，值越高越随机
    top_p=0.9,           # 核采样，保留概率质量前90%的词汇
    max_tokens=512,      # 最大生成长度
    repetition_penalty=1.1  # 重复惩罚，避免重复生成
)

# 单个提示词生成
prompt = "请介绍一下人工智能的发展历程"
print(f"\n输入提示词: {prompt}")
print("正在生成回复...")

start_time = time.time()
outputs = llm.generate([prompt], sampling_params)
generate_time = time.time() - start_time

generated_text = outputs[0].outputs[0].text
print(f"\n生成结果: {generated_text}")
print(f"生成耗时: {generate_time:.2f}秒")
print(f"生成长度: {len(generated_text)}字符")

INFO 08-21 15:44:09 [__init__.py:241] Automatically detected platform cuda.
正在加载ERNIE-4.5-0.3B模型...
INFO 08-21 15:44:10 [utils.py:326] non-default args: {'model': 'baidu/ERNIE-4.5-0.3B-PT', 'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 2048, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json: 0.00B [00:00, ?B/s]

INFO 08-21 15:44:16 [__init__.py:711] Resolved architecture: Ernie4_5ForCausalLM
WARNING 08-21 15:44:16 [__init__.py:2819] Casting torch.bfloat16 to torch.float16.
INFO 08-21 15:44:16 [__init__.py:1750] Using max model len 2048
INFO 08-21 15:44:18 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████████████████████| 67/67 [00:01<00:00, 64.09it/s]


(EngineCore_0 pid=2440) INFO 08-21 15:54:29 [gpu_model_runner.py:2708] Graph capturing finished in 1 secs, took 0.81 GiB
(EngineCore_0 pid=2440) INFO 08-21 15:54:29 [core.py:214] init engine (profile, create kv cache, warmup model) took 21.78 seconds
INFO 08-21 15:54:30 [llm.py:298] Supported_tasks: ['generate']
模型加载完成，耗时: 620.29秒

输入提示词: 请介绍一下人工智能的发展历程
正在生成回复...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                      | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…


生成结果: 。
<|SpecialToken:PPL|>
**第一阶段：萌芽期（1950s-1970s）**
- **起步阶段**：1956年，美国海军的“飞行实验机”Dornier 224原型机成功完成首次空中加油任务。这一事件标志着AI技术从理论走向实践，为后续发展奠定了基础。
- **早期应用**：1958年，IBM公司开发出了首个基于逻辑的计算机程序——“智能系统”（IBM Intelligent System），它通过分析大量数据和执行复杂的算法来解决问题，开创了AI的应用领域。
- **技术突破**：1963年，“贝尔实验室”的罗杰·格里菲斯提出“规则基达”理论，解释AI的本质是模拟人类智能的行为模式，为后来的深度学习等模型奠定了理论基础。
**第二阶段：爆发期（1980s-2000s）**
- **机器学习兴起**：1970年代末，斯坦福大学教授理查德·米勒等人提出了著名的“贝叶斯网络”，将概率论与统计方法相结合，使机器学习成为可能。
- **自然语言处理（NLP）诞生**：1970年，艾伦·图灵发表《论计算机可理解性》，引发了对AI理解能力的巨大挑战，推动了NLP技术的快速发展。
- **大规模数据处理**：1986年，IBM发布了世界上第一款大规模数据处理工具——“批处理系统”——IBM Rational Software Package，实现了海量数据的快速分析和处理。
- **深度学习崛起**：1997年，Deep Learning被正式纳入标准规范，标志着AI领域的又一次重大变革，使得机器可以像人一样思考和学习。
**第三阶段：成熟期（2000s至今）**
- **强化学习（RL）进入实用化阶段**：20世纪90年代后期，阿尔法狗战胜了国际象棋冠军，证明了强化学习技术在机器人领域的应用前景。这一突破标志着AI在各种复杂场景中的实际应用达到了新的高度。
- **脑科学与神经科学深入融合**：近年来，随着脑成像技术的发展，研究人员发现大脑具有类似于人类的认知功能，如图像识别、语言理解等，开启了AI与神经科学的深度交叉研究。
- **智能家居普及**：智能家居设备开始融入日常生活，推动了AI与物联网技术的结合
生成耗时: 1.69秒
生成长度: 923字符


### 对话模式生成

In [25]:
# 对话模式示例
def format_conversation(messages):
    """格式化对话消息为模型可理解的格式"""
    conversation = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        if role == "user":
            conversation += f"用户: {content}\n"
        elif role == "assistant":
            conversation += f"助手: {content}\n"
        elif role == "system":
            conversation += f"系统: {content}\n"
    conversation += "助手: "  # 提示模型开始回复
    return conversation

# 对话示例
messages = [
    # {"role": "system", "content": "你是一个有用的AI助手，请用中文回答问题。"},
    {"role": "user", "content": "你好，请介绍一下自己"}
]

conversation_prompt = format_conversation(messages)
print(f"对话提示词:\n{conversation_prompt}")

# 生成对话回复
outputs = llm.generate([conversation_prompt], sampling_params)
response = outputs[0].outputs[0].text

print(f"\n对话回复: {response}")

对话提示词:
用户: 你好，请介绍一下自己
助手: 


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                      | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…


对话回复: 您好！我是百度研发AI的智能助手，名叫文心一言。我的名字里藏着“智”和“语”，既代表语言能力又暗示着我对知识、信息和社会问题的深刻理解。很高兴能为您提供帮助。


## 启动vLLM服务

### 基础服务启动

```bash
# 启动vLLM服务器（基础配置）
vllm serve baidu/ERNIE-4.5-0.3B-PT \
    --host 0.0.0.0 \
    --port 8000 \
    --trust-remote-code
```

### 命令行操作

#### 1. 检查服务状态

启动服务后，可以通过以下命令检查服务是否正常运行：

```bash
# 检查模型列表
curl http://localhost:8000/v1/models
```

#### 2. 使用Completions API

```bash
# 基础文本补全
curl http://localhost:8000/v1/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "baidu/ERNIE-4.5-0.3B-PT",
        "prompt": "人工智能的未来发展趋势是",
        "max_tokens": 256,
        "temperature": 0.7
    }'
```

#### 3. 使用Chat Completions API

```bash
# 聊天对话API
curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "baidu/ERNIE-4.5-0.3B-PT",
        "messages": [
            {"role": "system", "content": "你是一个有用的AI助手。"},
            {"role": "user", "content": "请介绍一下Python编程语言"}
        ],
        "max_tokens": 512,
        "temperature": 0.7
    }'
```

#### 4. 高级服务启动选项

```bash
# 启动服务器（包含API密钥验证）
vllm serve baidu/ERNIE-4.5-0.3B-PT \
    --host 0.0.0.0 \
    --port 8000 \
    --trust-remote-code \
    --api-key your-api-key-here

# 启动服务器（指定GPU和内存配置）
vllm serve baidu/ERNIE-4.5-0.3B-PT \
    --host 0.0.0.0 \
    --port 8000 \
    --trust-remote-code \
    --gpu-memory-utilization 0.8 \
    --max-model-len 4096

# 启动服务器（使用ModelScope下载模型）
export VLLM_USE_MODELSCOPE=True
vllm serve baidu/ERNIE-4.5-0.3B-PT \
    --host 0.0.0.0 \
    --port 8000 \
    --trust-remote-code
```

#### 5. 服务管理命令

```bash
# 查看vLLM帮助信息
vllm serve --help

# 检查vLLM版本
vllm --version

# 停止服务（如果在前台运行，使用Ctrl+C）
# 如果在后台运行，可以使用以下命令查找并停止进程
ps aux | grep vllm
kill <process_id>
```

#### 6. 测试连接

```bash
# 简单的健康检查
curl -f http://localhost:8000/health || echo "服务未启动"

# 测试基础响应
curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "baidu/ERNIE-4.5-0.3B-PT",
        "messages": [{"role": "user", "content": "你好"}],
        "max_tokens": 50
    }' | jq '.choices[0].message.content'
```

> **注意**：
> - 确保已安装vLLM：`pip install vllm`
> - 服务启动可能需要几分钟时间来加载模型
> - 建议在生产环境中使用进程管理工具（如systemd、supervisor等）来管理vLLM服务

## 参考资源

- [vLLM官方文档](https://docs.vllm.ai/) <mcreference link="https://docs.vllm.ai/" index="1">1</mcreference>
- [ERNIE-4.5-0.3B模型页面](https://huggingface.co/baidu/ERNIE-4.5-0.3B-PT) <mcreference link="https://huggingface.co/baidu/ERNIE-4.5-0.3B-PT" index="2">2</mcreference>
- [ERNIE-4.5官方博客](https://yiyan.baidu.com/blog/posts/ernie4.5/) <mcreference link="https://yiyan.baidu.com/blog/posts/ernie4.5/" index="1">1</mcreference>
- [vLLM ERNIE支持PR](https://github.com/vllm-project/vllm/pull/20220) <mcreference link="https://github.com/vllm-project/vllm/pull/20220" index="4">4</mcreference>
- [OpenAI API文档](https://platform.openai.com/docs/api-reference)

---

*注意：ERNIE-4.5模型的vLLM支持可能需要最新版本的vLLM。如果遇到兼容性问题，请检查vLLM版本或参考官方文档获取最新信息。*